In [96]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [97]:
# 지니 차트 200위까지 크롤링
# 반복문을 사용하여 크롤링
# 수집한 데이터를 데이터 프레임으로 저장
# 엑셀로 내보내기
# 과제는 github에 제출

In [98]:
site = 'https://www.genie.co.kr/chart/top200'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
request = requests.get(site, headers=header)
print(request)

<Response [200]>


In [99]:
soup = BeautifulSoup(request.text)

In [100]:
div = soup.find('div', {'class': 'rank-page-nav'})

In [101]:
links = div.findAll('a')

In [102]:
for link in links:
    print(link['href'])

?ditc=D&ymd=20230601&hh=16&rtm=Y&pg=1
?ditc=D&ymd=20230601&hh=16&rtm=Y&pg=2
?ditc=D&ymd=20230601&hh=16&rtm=Y&pg=3
?ditc=D&ymd=20230601&hh=16&rtm=Y&pg=4


In [105]:
def genie_chart(link):
    data = []
    site = f'https://www.genie.co.kr/chart/top200{link}'
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
    request = requests.get(site, headers=header)
    soup = BeautifulSoup(request.text)

    ranks = soup.findAll('td', {'class': "number"})
    titles = soup.findAll('a', {'class': "title ellipsis"})
    info = soup.find('tbody')

    artists = info.findAll('a', {'class': "artist"})
    albumtitles = soup.findAll('a', {'class': "albumtitle"})

    for r, t, art, album in zip(ranks, titles, artists, albumtitles):
        rank = r.text.strip().split('\n')[0]
        title = t.text.strip()
        artist = art.text.strip()
        albumtitle = album.text.strip()

        data.append({'rank' : rank, 'title' : title, 'artist' : artist, 'albumtitle' : albumtitle})

    return data

In [106]:
datas = []
for link in links:
    url = link['href']
    result = genie_chart(url)
    # print(result)
    datas += result

In [107]:
df = pd.DataFrame(datas)

In [109]:
df.head(7)

,rank,title,artist,albumtitle
0,1,퀸카 (Queencard),(여자)아이들,I feel
1,2,I AM,IVE (아이브),I've IVE
2,3,Spicy,aespa,MY WORLD - The 3rd Mini Album
3,4,Kitsch,IVE (아이브),I've IVE
4,5,UNFORGIVEN (Feat. Nile Rodgers),LE SSERAFIM (르세라핌),UNFORGIVEN
5,6,꽃,지수 (JISOO),ME
6,7,Hype boy,NewJeans,NewJeans 1st EP 'New Jeans'


In [110]:
df.to_excel('genie_chart.xlsx')